![snap](https://media4.giphy.com/media/Sb7WSbjHFNIL6/200.gif)

# Run a remote training job

## What you will learn in this course 🧐🧐

Let's now experience the power of MLFlow. One of the main perks of packaging your training script is that you can run it on remote servers. In this course we will learn:

* How to run a remote training job on a EC2 Instance

## Project

For this course, we want to train an `sklearn` Random Forest and run a `GridSearch`. As you already know, this can be quite computation heavy. Therefore it makes it a perfect usecase for remote training! 

Here is what our project looks like:

```
├── MLproject
├── secrets.sh
└── train.py
```

👉 Check out the source code here: https://github.com/antoinekrajnc/mlflow_101 👈

> 👋 You can also clone that repository: `git clone https://github.com/antoinekrajnc/mlflow_101`

## Remote training process 

Now that you are familiar with the code, let us explain what we need to do to run a remote training job: 

1. We need to setup an EC2 instance where we need to: 
    * Choose lots of CPUs & RAM 
    * Install `docker`,  `git` and `mlflow` 

2. Set environment variables 

3. Run the project (`mlflow run PROJECT_URI`)

Let's walk you through these steps one by one

## **Step-1**: Setup EC2 Instance 

### Launch EC2 

Let's first setup our EC2. If you feel a little rusty on how to setup an EC2, simply go back to our course **Introduction_to_EC2**. The higher CPU & RAM you choose the faster the training job. However checkout pricing before so that you don't get surprised! 

* [**EC2 Pricing List**](https://aws.amazon.com/fr/ec2/pricing/on-demand/)

In this example, we will choose a `g4dn.8xlarge` Amazon Linux instance with:

* 32 CPUs
* 128 Go RAM 

This is basically a rocket computer 🚀

### Configurations
 
Before launching your EC2 instance though, go to *Configure Instance* > *User Data* and paste the following code:

NB: Note that the initial #! to indicate EC2 that we are giving it a shell script to execute, you may find more specific info about this in the [AWS documentation](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/user-data.html#user-data-shell-scripts)

```bash
#!/bin/bash
sudo yum update -y
sudo yum install docker -y
echo -e "sudo service docker start" >> .bashrc
sudo usermod -a -G docker ec2-user
sudo yum install git -y
sudo yum install python3 -y
sudo yum install python3-pip -y
pip3 install mlflow
```

Just like this:

![crack](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/EC2_Remote_tracking_setup.png)

This code will be run when the instance will boot up. It simply:

* Updates all currently installed packages 
* installs `docker` 
* turns on `docker` each time you connect to the instance
* gives `ec2-user` sudo privileges 
* installs `git` 
* installs `python`
* installs `pip`
* installs `mlflow`

> 👋 Don't worry, you don't need to remember this code by heart, you will go back to it when necessary! 

### Allow SSH

Make sure you allowed SSH in your security group and you can launch your machine!


### **Reboot your machine** 

As a final config, **you need to reboot your machine** as you ran commands that gave sudo privileges to `ec2-user`:

![crack](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/EC2_Reboot.png)

## **Step-2**: Set environment variables 

### Securely send files to your EC2 instance
Your EC2 instance is a new machine that you can connect to from your local computer, which means that by default any file that's on your local computer may not be accessed from the EC2 instance and vice versa. If you wish to send some file to your EC2 instance (for example a `secrets.sh` file with your environment variables) you may do so using a command like this:

```shell
scp -i name_of_your_ec2_authentication_file.pem name_of_the_file_you_want_to_send ec2-user@ec2-00-00-000-000.eu-west-3.compute.amazonaws.com:~/.
```

Note that you will need to modify the ec2 access point according the info in the ssh connect information page.

![crack](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/EC2_SSH_connect.png)


### SSH Into your instance 

Now you need to set your environment variables that you wrote in your `MLProject` file. In our case we have: 

*   `MLFLOW_TRACKING_URI`
*   `AWS_ACCESS_KEY_ID`
*   `AWS_SECRET_ACCESS_KEY`
*   `BACKEND_STORE_URI`
*   `ARTIFACT_ROOT`
* `MLFLOW_EXPERIMENT_NAME`

What you can do is to locally create a `secrets.sh` file containing the following commands:

```bash
export MLFLOW_TRACKING_URI=REPLACE_ME_WITH_YOUR_MLFLOW_TRACKING_URI
export AWS_ACCESS_KEY_ID=REPLACE_ME_WITH_YOUR_AWS_ACCESS_KEY_ID
export AWS_SECRET_ACCESS_KEY=REPLACE_ME_WITH_YOUR_AWS_SECRET_ACCESS_KEY
export BACKEND_STORE_URI=REPLACE_ME_WITH_YOUR_BACKEND_STORE_URI
export ARTIFACT_ROOT=REPLACE_ME_WITH_YOUR_ARTIFACT_ROOT
export MLFLOW_EXPERIMENT_NAME=REPLACE_ME_WITH_YOUR_EXPERIMENT_NAME
```

Send it to your ec2 instance using `scp`.

Ok now let's SSH into your instance. You can do it through your local terminal with a `.pem` file or you can SSH directly using Amazon GUI. Simply click on *Connect* 

![crack](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/EC2_SSH_connect.png)

> 👋 **IMPORTANT** Make sure that your user is set `ec2-user`

Then run `source secrets.sh`. 

> 👋 you can verify that everything works correctly by running `echo $MLFLOW_TRACKING_URI` if you see your remote server URL appearing. It means that everything worked correctly. 

## **Step-3**: Run the project

Now the easiest way to run your project is to host it on a Github Repository and then, in your remote shell run: 


`mlflow run GITHUB_URL --build-image`

The `--build-image` option is only valid for Docker projects. If specified, it will build a new Docker image that’s based on the image specified by the image field in the MLproject file, and contains files in the project directory.

If the project is not running, it's probably because EC2 does not have access to other AWS services that let it use docker properly.

And that's it! 😮

If you want to try it out, you can use this course's Github URL - `https://github.com/antoinekrajnc/mlflow_101`

> 👋 If you feel a little rusty on Git & Github, feel free to go back to our [Git & Github course](https://app.jedha.co/course/git-and-github-ft/git-and-github-ft-replay) 


## Results 

Now lean back on your chair and watch your computer do the job! If you didn't have any errors, you will see some logs on your MLFlow remote server. In our case, the hyperparameter tuning took **8.4 min** on our `g4dn.8xlarge` against **48.6 min** on our local computer!! 

![crack](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/remote_training_experience.png)

## Troubleshooting 

Computers can be grumpy and therefore, you might encounter some bugs. To help you get through them, here is a list of verifications you can run: 

### scp file transfer is not working
Type the following command in your local machine's console (make sure the `.pem` key is in your working directory):

```bash
chmod 400 your_key.pem
```

### Make sure installations were made

First, there might be a problem with installations. It can be dependencies or with `docker`. To verify that everything has been installed, simply run: 

`mlflow --version` 

You should see a version number for mlflow appearing. If you see something like `mlflow is not a command` it means that it hasn't been installed. 

To check whether docker is installed, simply run: 

`sudo docker info`

You should again see some information coming up about Docker.

If some packages are not installed, you can copy-paste the following commands in the EC2 terminal and then **reboot** your EC2 instance:

```bash
#!/bin/bash
sudo yum update -y
sudo yum install docker -y
echo -e "sudo service docker start" >> .bashrc
sudo usermod -a -G docker ec2-user
sudo yum install git -y
sudo yum install python3 -y
sudo yum install python3-pip -y
pip3 install mlflow
```

### Docker is not turned on

Another verification that you need to run is whether `docker` is turned on. TO verify that, you should run:

`sudo docker info` 

If you see an output like this:

```
Client:
 Context:    default
 Debug Mode: false
 Plugins:
  app: Docker App (Docker Inc., v0.9.1-beta3)
  buildx: Build with BuildKit (Docker Inc., v0.5.1-docker)
  compose: Docker Compose (Docker Inc., 2.0.0-beta.1)
  scan: Docker Scan (Docker Inc., v0.8.0)

Server:
 Containers: 9
  Running: 0
  Paused: 0
  Stopped: 9
 Images: 29
 Server Version: 20.10.6
...
 HTTP Proxy: http.docker.internal:3128
 HTTPS Proxy: http.docker.internal:3128
 Registry: https://index.docker.io/v1/
 Labels:
 Experimental: false
 Insecure Registries:
  127.0.0.0/8
 Live Restore Enabled: false
 ```

 It means that it is turned on. If you don't see it, simply run:

 `sudo service docker start`

### Sudo is still necessary for your user 

Finally, one final verification you can do is to check whether your user has `sudo` privilege, meaning you don't have to `sudo COMMMAND` everything. Especially you need to be able to run `docker` directly without `sudo`. 

To verify that, simply run: 

`docker info`

If you see an output like `Permission denied`, it means that your user doesn't have `sudo` privilege and you need to grant it. To do so, run:

`sudo usermod -a -G docker REPLACE_BY_YOUR_USERNAME`

then you need to **reboot** your instance and it should work. 

> 👋 In the tutorial, our username was `ec2-user` which is usually the default username. To verify that it is yours too, simply search for it here when you SSH to your instance: 

![crack](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/EC2_username.png)

## Resources 📚📚

* [EC2 Pricing List](https://aws.amazon.com/fr/ec2/pricing/on-demand/)
* [How to run a command at EC2 Instance RE-boot?](https://stackoverflow.com/questions/70582211/how-to-run-a-command-at-ec2-instance-re-boot?noredirect=1#comment124780237_70582211)